# Project 2
## Team B, CA Beaches


In [69]:
# set environment
from bs4 import BeautifulSoup
from datetime import date
import pandas as pd
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [48]:
from sqlalchemy import create_engine, insert
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
#from config import password

In [53]:
# initialize connection with Chrome Driver
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=True)

[WDM] - ====== WebDriver manager ======
[WDM] - Current google-chrome version is 89.0.4389
[WDM] - Get LATEST driver version for 89.0.4389


[WDM] - Driver [C:\Users\kate_\.wdm\drivers\chromedriver\win32\89.0.4389.23\chromedriver.exe] found in cache


In [54]:
# define our base URL
base_url = "https://www.californiabeaches.com/beaches/"

In [55]:
# scrape the regions off the main beach page
browser.visit(base_url)
html = browser.html
soup = BeautifulSoup(html, "html.parser") 

cali_soup = soup.find(id="regions")


In [56]:
# initialize empty county URL list
county_urls = []    
county_url = ""

# scrape lists of counties in each region
region_soup = cali_soup.find_all("ul")


for region in region_soup:
    county_soup = region.find_all("a", href=True)


    for county in county_soup:
        if county:
            # we found one, so pull out the URL
            new_url = county["href"]
            
            # only save one copy of each url
            if new_url != county_url:
                county_url = new_url
                x = county_url.split("/")
                
                # split out the region and county
                # and clean out hyphens
                region = x[-3]
                cnty = x[-2]
                cnty = cnty.replace("-", " ")
                county_urls.append([region.title(), cnty.title(), county_url])
                

print(len(county_urls))


16


In [58]:
# initialize empty area list
area_urls = []    
area_url = ""

# loop through county URLs
for county in county_urls:

    # scrape the county webpage
    browser.visit(county[2])
    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    
    # scrape the list of beaches
    county_soup = soup.find(id="beach-list")
    
    # scrape the links
    area_soup = county_soup.find_all("a", href=True)
    
    for area in area_soup:
        if area:
            # we have one, so pull out the URL
            new_url = area["href"]

            if new_url != area_url:
                area_url = new_url
                x = area_url.split("/")
                
                # split out the area name
                # and clean out hyphens
                curr_area = x[-2]
                curr_area = curr_area.replace("-", " ")
                area_urls.append([county[0], county[1], curr_area.title(), area_url])
                
#print(area_urls)
print(len(area_urls))

112


In [60]:
# initialize empty beach list
beach_urls = []    
beach_url = ""

for area in area_urls:
    
    # scrape the area page
    browser.visit(area[3])
    html = browser.html
    soup = BeautifulSoup(html, "html.parser")
    
    # scrape the list of beaches
    area_soup = soup.find(id="beach-list")
    
    # scrape the links
    beach_soup = area_soup.find_all("a", href=True)
    
    for beach in beach_soup:
        if beach:

            # we found one, so pull out the URL
            new_url = beach["href"]

            if new_url != beach_url:
                beach_url = new_url
                x = beach_url.split("/")
                # split out the beach name and
                # clean out the hyphens
                curr_beach = x[-2]
                curr_beach = curr_beach.replace("-", " ")
                beach_urls.append([area[0], area[1], area[2], curr_beach.title(), beach_url])
                
print(beach_urls[0])
print(len(beach_urls))


['Southern', 'San Diego County', 'Carlsbad', 'Carlsbad City Beach', 'https://www.californiabeaches.com/beach/carlsbad-city-beach/']
1014


In [62]:
# initialize empty list of data titles
title_list = []


for beach in beach_urls:
    
    # append an empty dictionary to each beach list
    # to hold beach info
    beach.append({})
    
    try:
        
        # scrape each beach page
        browser.visit(beach[4])
        html = browser.html
        soup = BeautifulSoup(html, "html.parser")

        # scrape the list of data
        data_soup = soup.find("dl")

        # scrape the titles and data values
        title_soup = data_soup.find_all("dt")
        value_soup = data_soup.find_all("dd")

        i = 0
        
        # save column titles and data
        for title in title_soup:
            
            if title.text == "Address":
                addr_str = str(value_soup[i])
                addr_br = addr_str.split("<br/>")
        
                addr1_br = addr_br[0].split(">")
                addr1 = addr1_br[-1]
        
                addr2_br = addr_br[1].split("<")
                addr2 = addr2_br[0]
        
                beach[5]["Address1"] = addr1
                beach[5]["Address2"] = addr2
                
                if "Address1" not in title_list:
                    title_list.append("Address1")
                    title_list.append("Address2")
        
            elif title.text == "Owner":
                owner = value_soup[i].text
                beach[5][title.text] = owner
            
                if value_soup[i].a:
                    owner_url = value_soup[i].a["href"]
                    beach[5]["Owner URL"] = owner_url
                    
                    if "Owner URL" not in title_list:
                        title_list.append("Owner URL")
        
            else:
                beach[5][title.text] = value_soup[i].text
                if title.text not in title_list:
                    title_list.append(title.text)
                    
                    
            i+=1


    except Exception as e:
        print(f"Error processing: {beach[4]}, {e}")

    
print(beach_urls[0])


['Southern', 'San Diego County', 'Carlsbad', 'Carlsbad City Beach', 'https://www.californiabeaches.com/beach/carlsbad-city-beach/', {'Address1': 'Ocean Street and Grand Avenue', 'Address2': 'Carlsbad, CA 92008', 'Park Name': 'Carlsbad State Beach', 'Owner': 'State Park', 'Owner URL': 'http://www.carlsbadca.gov/about/visitorinfo/Pages/about-carlsbad-beaches1.aspx', 'Activities': 'Sunbathing, Beach Walking', 'Amenities': 'No Facilities', 'Pet Policy': 'No dogs allowed on the beach or sea wall', 'Fees': 'Free street parking'}]


In [64]:
# define static portion of beach dictionary
beach_data = {"Region": [], "County": [], "Area": [], "Beach Name": [], "Beach URL": []}

# loop through list of all titles found across all beaches
# and add a dictionary to the beach data
for title in title_list:
    beach_data[title] = []

# loop through all the beaches we scraped
for beach in beach_urls:
    
    # add data to appropriate lists
    beach_data["Region"].append(beach[0])
    beach_data["County"].append(beach[1])
    beach_data["Area"].append(beach[2])
    beach_data["Beach Name"].append(beach[3])
    beach_data["Beach URL"].append(beach[4])
    
    for title in title_list:
        if title in beach[5]:
            beach_data[title].append(beach[5][title])
        else:    
            beach_data[title].append("")
        
    

In [66]:
# dump data into dataframe
beach_df = pd.DataFrame(beach_data)

In [67]:
# display dataframe
beach_df

,Region,County,Area,Beach Name,Beach URL,Address1,Address2,Park Name,Owner URL,Activities,Amenities,Pet Policy,Fees,Phone,Other Names
0,Southern,San Diego County,Carlsbad,Carlsbad City Beach,https://www.californiabeaches.com/beach/carlsb...,Ocean Street and Grand Avenue,"Carlsbad, CA 92008",Carlsbad State Beach,http://www.carlsbadca.gov/about/visitorinfo/Pa...,"Sunbathing, Beach Walking",No Facilities,No dogs allowed on the beach or sea wall,Free street parking,,
1,Southern,San Diego County,Carlsbad,Carlsbad Lagoon Agua Hedionda,https://www.californiabeaches.com/beach/carlsb...,4750 Bayshore Dr,"Carlsbad, CA 92008",Public Lagoon,https://www.carlsbadca.gov/residents/fun/lagoo...,"Kayaking, Canoeing, Stand-Up Paddleboarding, B...","Lagoon, Trails, Rentals",Leashed dogs ok on land but not in the water,Free street parking,760-434-3089,
2,Southern,San Diego County,Carlsbad,North Ponto Beach,https://www.californiabeaches.com/beach/north-...,Carlsbad Blvd and Island Way,"Carlsbad, CA 92011",South Carlsbad State Beach,http://www.parks.ca.gov/?page_id=660,"Surfing, Swimming, Fishing, Scuba Diving, Sunb...","Restrooms, Lifeguard",No dogs allowed on the beach,Fee for parking,,
3,Southern,San Diego County,Carlsbad,Robert Frazee State Beach,https://www.californiabeaches.com/beach/robert...,3150 Ocean Street,"Carlsbad, CA 92008",Carlsbad State Beach,http://www.parks.ca.gov/?page_id=653,"Surfing, Swimming, Sunbathing, Fishing, Runnin...","Restrooms, Showers, Lifeguard, Grass Park, Ben...",No dogs allowed on the beach,,,
4,Southern,San Diego County,Carlsbad,South Carlsbad State Beach,https://www.californiabeaches.com/beach/south-...,Carlsbad Blvd and Poinsettia Ln,"Carlsbad, CA 92011","South Carlsbad State Beach, Batiquitos Lagoon ...",http://www.parks.ca.gov/?page_id=660,"Camping, Surfing, Swimming, Scuba Diving, Hiki...","Campground, Fire Pits, Restrooms, Showers, Lif...",No dogs allowed on the beach,"Fee for camping, free day-use parking south of...",(760) 438-3143,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1009,Northern,Del Norte County,Klamath,Lagoon Creek Beach,https://www.californiabeaches.com/beach/lagoon...,Redwood Hwy,"Klamath, CA 95548","Redwood National Park, Lagoon Creek Picnic Area",http://www.nps.gov/redw/,"Hiking, Beachcombing, Birdwatching, Surfing, F...","Picnic Tables, Restrooms, Driftwood, Trails, W...","Dogs allowed on leash on beach, but not allowe...",Free parking,,False Klamath Cove
1010,Northern,Del Norte County,Klamath,Wilson Creek Beach,https://www.californiabeaches.com/beach/wilson...,Redwood Hwy and Wilson Creek Rd,"Klamath, CA 95548",Del Norte Coast Redwoods State Park,http://www.parks.ca.gov/?page_id=414,"Tidepooling, Birdwatching, Surfing, Beach Walking","Picnic Tables, BBQs, Tide Pools, Driftwood, Creek",Dogs allowed on leash,Free parking,,Wilson Beach
1011,Northern,Del Norte County,Smith River,Clifford Kamph Memorial Park,https://www.californiabeaches.com/beach/cliffo...,15100 Hwy 101,"Smith River, CA 95567",Clifford Kamph Memorial Park,http://www.co.del-norte.ca.us/departments/parks,"Beachcombing, Hiking, Surfing, Fishing, Campin...","Campground, Restrooms, Picnic Tables, BBQs, Tr...",Dogs allowed on leash,"Fee for camping, free day-use parking",707-464-7230,
1012,Northern,Del Norte County,Smith River,Pelican State Beach,https://www.californiabeaches.com/beach/pelica...,17200 Hwy 101,"Smith River, CA 95567",Pelican State Beach,http://www.parks.ca.gov/?page_id=412,"Beachcombing, Beach Walking, Fishing, Surfing,...","Driftwood, No Facilities",No dogs allowed on the beach,Free parking,,


In [68]:
# write dataframe to a CSV file
beach_df.to_csv("data/beach_info.csv")